In [1]:
import sliceoptim.core as core
import pandas as pd
from skopt.space import Real, Integer
from skopt import Space, Optimizer
from numpy import around, NaN
import numpy as np

test_stl = "../assets/calicat.stl"

In [3]:
pd.read_csv("results_test.csv", index_col = 0)

,first-layer-temperature,first-layer-bed-temperature,temperature,bed-temperature,first-layer-speed,first-layer-height,first-layer-extrusion-width,extrusion-multiplier,layer-height,print_time,quality,cost
0,218,47,203,55,43,0.334733,0.819606,1.034363,0.150993,35.682967,NaN,NaN
1,193,49,211,47,50,0.372197,0.974377,0.924020,0.397101,19.746391,NaN,NaN
2,198,53,219,50,23,0.300837,0.400491,0.859496,0.272207,22.225243,NaN,NaN


In [4]:

printer = core.Printer(
    name="test_printer",
    bed_size=[220, 220], nozzle_diameter=0.4, max_speed=120, min_speed=5
)
filament = core.Filament(
    name="test_fil",
    kind="pla",
    extrusion_temp_range=[180, 250],
    bed_temp_range=[25, 80],
    diameter=1.75,
)



params = []
params.append(Integer(name="first-layer-temperature", low=190, high=220))
params.append(Integer(name="first-layer-bed-temperature", low=40, high=55))
params.append(Integer(name="temperature", low=190, high=220))
params.append(Integer(name="bed-temperature", low=40, high=55))
params.append(
    Integer(
        name="first-layer-speed",
        low=printer.min_speed,
        high=printer.max_speed * 0.5,
    )
)
params.append(
    Real(
        name="first-layer-height",
        low=printer.nozzle_diameter * 0.75,
        high=printer.nozzle_diameter,
    )
)
params.append(
    Real(
        name="first-layer-extrusion-width",
        low=printer.nozzle_diameter * 0.9,
        high=printer.nozzle_diameter * 2.5,
    )
)
params.append(Real(name="extrusion-multiplier", low=0.8, high=1.2))
params.append(
    Real(
        name="layer-height",
        low=printer.nozzle_diameter * 0.2,
        high=printer.nozzle_diameter,
    )
)
space = Space(params)


out = "test.gcode"

stl = test_stl
is_fl = False
s = 30
exp = core.Experiment(
    "test",
    sample_file=stl,
    is_first_layer=is_fl,
    spacing=s,
    printer=printer,
    filament=filament,
    params_space=space,
    output_file=out,
)

In [5]:
space.dimension_names

['first-layer-temperature',
 'first-layer-bed-temperature',
 'temperature',
 'bed-temperature',
 'first-layer-speed',
 'first-layer-height',
 'first-layer-extrusion-width',
 'extrusion-multiplier',
 'layer-height']

In [8]:
exp.create_new_sample_grid(3)

In [11]:
exp.export_csv("")

In [4]:
samples = exp.get_samples_list()
for s in samples:
    s.quality = np.random.randn()

In [5]:
exp.compute_and_update_samples_costs()

In [6]:
exp.get_samples_results_df()

,print_time,quality,cost
0,30.594644,-0.552596,10.0
1,20.484318,-0.285078,5.0
2,44.500176,0.732811,0.0


In [7]:
exp.get_samples_results_df().columns.to_list()

['print_time', 'quality', 'cost']

In [7]:
exp.create_new_sample_grid(3)
samples = exp.get_samples_list()
for s in samples:
    s.quality = np.random.randn()
exp.compute_and_update_samples_costs()

In [8]:
exp.get_designs()

,first-layer-temperature,first-layer-bed-temperature,temperature,bed-temperature,first-layer-speed,first-layer-height,first-layer-extrusion-width,extrusion-multiplier,layer-height
0,194,46,198,49,54,0.399056,0.698616,0.814038,0.178187
1,208,50,191,40,41,0.310453,0.403733,0.957373,0.313139
2,216,51,218,54,20,0.373471,0.595718,1.104605,0.119048
0,194,46,198,49,54,0.399056,0.698616,0.814038,0.178187
1,208,50,191,40,41,0.310453,0.403733,0.957373,0.313139
2,216,51,218,54,20,0.373471,0.595718,1.104605,0.119048


In [9]:
exp.register_results_to_optimizer()

AttributeError: 'Experiment' object has no attribute 'register_results_to_optimizer'

In [10]:
exp.estim_best_config()

({'first-layer-temperature': 192.0,
  'first-layer-bed-temperature': 53.0,
  'temperature': 209.0,
  'bed-temperature': 46.0,
  'first-layer-speed': 11.0,
  'first-layer-height': 0.31062114999987145,
  'first-layer-extrusion-width': 0.3640635783459852,
  'extrusion-multiplier': 1.0898419636084324,
  'layer-height': 0.2669731284399158},
 array([0.950981]))

In [27]:
exp.optimizer.yi

[5.0, 10.0, 0.0, 5.0, 10.0, 0.0]

In [29]:
exp.optimizer.n_initial_points_

4

In [11]:
exp.get_samples_list()[0].get_info()

{'filename': 'calicat.stl',
 'size_x': 28.535534,
 'size_y': 28.5,
 'size_z': 35.0,
 'min_x': 0.464466,
 'min_y': 0.0,
 'min_z': 0.0,
 'max_x': 29.0,
 'max_y': 28.5,
 'max_z': 35.0,
 'number_of_facets': 876.0,
 'manifold': 'yes',
 'number_of_parts': 1.0,
 'volume': 12501.378906}